# Context

In this notebook we will do some data cleaning and data preprocessing in order to fix some issues of the data and prepare it to further analysis and modelling.

# Load packages

In [596]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random
import os
from unidecode import unidecode  # Library to remove accents


pd.options.display.max_columns = 100
pd.options.display.max_rows = 100


# Load data

using relative paths

In [597]:
filename = "data_cleaning.ipynb" # Current file name
print(f'Current file name: {filename}\n')
print(f'Current absolute path: {os.getcwd()}')

# Specify the paths, relative to the current file
NOTEBOOKS_DIR = os.path.dirname(os.path.abspath(filename))
BASE_DIR = os.path.dirname(NOTEBOOKS_DIR)
DATA_DIR = os.path.join(BASE_DIR, "Data")
OUTPUT_DIR = os.path.join(DATA_DIR, "output_data")

print(f'BASE_DIR: {BASE_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'OUTPUT_DIR: {OUTPUT_DIR}')

Current file name: data_cleaning.ipynb

Current absolute path: c:\Users\jhorea\Dropbox\ASPECTOS MAESTRIA\Retos_maestria\Reto_bancow\Analitica_1\notebooks
BASE_DIR: c:\Users\jhorea\Dropbox\ASPECTOS MAESTRIA\Retos_maestria\Reto_bancow\Analitica_1
DATA_DIR: c:\Users\jhorea\Dropbox\ASPECTOS MAESTRIA\Retos_maestria\Reto_bancow\Analitica_1\Data
OUTPUT_DIR: c:\Users\jhorea\Dropbox\ASPECTOS MAESTRIA\Retos_maestria\Reto_bancow\Analitica_1\Data\output_data


In [598]:
df_base = pd.read_excel(os.path.join(DATA_DIR, "historico_hallazgos.xlsx"))
df_base.head(2)

,Num,ID_modif,Cliente,Analista,Nombre analista,Tipo crédito,Fecha desem,Visita_analista_crédito,Visita_auditor,Actividad,Monto,Cuota,Plazo,Categoria,Hallazgo,Tipo hallazgo,Riesgo,Calificación cartera,Relaciones Laborales,Oficina,zona,Regional,Validación unico,Tipo analisis,Clasificac analisis,Estado,Año,Tipo
0,1,7252440,Hector Julio Pabon Castano,AIZ,Juan Alejandro Trujillo Garcia,Renovacion,2021-06-11 00:00:00,NaN,2022-09-20 00:00:00,don patacon postobon,15069.268,792.642,36,FRAUDE_Y_PRÁCTICAS_INDEBIDAS,Crédito otorgado a mas de un titular con el mi...,Acto Irregular,1 - Alto,Si,Si,Dosquebradas,12,4,NaN,NaN,NaN,Cancelada,2022,Afecta Estabilidad
1,2,1143936676,Casas Marisol ...,JQQ,Alexander Joaqui Quintero,Renovacion,2021-06-15 00:00:00,2021-06-08 00:00:00,NaN,expendio de comidas preparadas en cafeterias ...,402.000,209.228,626,FRAUDE_Y_PRÁCTICAS_INDEBIDAS,Presunto negocio inexistente,Acto Irregular,1 - Alto,No,Si,Poblado,1,1,NaN,NaN,NaN,Castigo,2022,Afecta Estabilidad


# Overall check & Fix

Reviewing the columns, we can gather valuable information about the data and the context, in order to know how to develop an EDA and ML project.  
We can also clean the columns data, in order to have valuable information.

In [599]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919 entries, 0 to 3918
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Num                      3919 non-null   int64  
 1   ID_modif                 3919 non-null   int64  
 2   Cliente                  3918 non-null   object 
 3   Analista                 3919 non-null   object 
 4   Nombre analista          3919 non-null   object 
 5   Tipo crédito             3717 non-null   object 
 6   Fecha desem              3916 non-null   object 
 7   Visita_analista_crédito  3119 non-null   object 
 8   Visita_auditor           3800 non-null   object 
 9   Actividad                3867 non-null   object 
 10  Monto                    3919 non-null   float64
 11  Cuota                    3919 non-null   object 
 12  Plazo                    3919 non-null   int64  
 13  Categoria                3919 non-null   object 
 14  Hallazgo                

--> In this context, isnt necessary to use columns like "Num" & "ID_modif" because they are not relevant for the analysis. So, we **will eliminate it**.


In [600]:
df_base = df_base.drop(columns=["Num", "ID_modif"])

## Cliente

In [601]:
num_dif = df_base.Cliente.nunique()
print(f'Number of different clients: {num_dif}')

temp = df_base.Cliente.value_counts()
temp = temp[temp > 1]
print(f'Number of clients with more than one record: {len(temp)}')
print(f'\nClients with more than one record:')
temp

Number of different clients: 3459
Number of clients with more than one record: 229

Clients with more than one record:


Jose Manuel Arroyo Benavides     7
Hector Angel Ochoa Caballero     6
Fidel Antonio Carranza Garcia    6
Enith  Garcia Vanegas            6
Maria Eugenia Velasco Causaya    6
                                ..
Daniel Andres Barrero Sabogal    2
Mireya  Solis Otero              2
Silvia  Canga Mosquera           2
Tomas Albeiro Zabaleta Serpa     2
Yurani Katerine Lozano Perez     2
Name: Cliente, Length: 229, dtype: int64

In [602]:
print("Example of one client with more than one record:\n")
df_base[df_base["Cliente"]== "Nelly  Munoz Vivas"]

Example of one client with more than one record:



,Cliente,Analista,Nombre analista,Tipo crédito,Fecha desem,Visita_analista_crédito,Visita_auditor,Actividad,Monto,Cuota,Plazo,Categoria,Hallazgo,Tipo hallazgo,Riesgo,Calificación cartera,Relaciones Laborales,Oficina,zona,Regional,Validación unico,Tipo analisis,Clasificac analisis,Estado,Año,Tipo
3471,Nelly Munoz Vivas,ROO,Robinson Ocoro Centeno,Preferenci,2024-04-15 00:00:00,2024-04-15 00:00:00,NaN,arrendamientos,5254.384,335.892,36,Otras_Políticas,Formato de vinculación con campos en blanco y/...,Incumple\n P & P,2 - Medio,No,No,Palmira,3,1,NaN,Análisis de carácter,Políticas,NaN,2024,NaN
3472,Nelly Munoz Vivas,ROO,Robinson Ocoro Centeno,Preferenci,2024-04-15 00:00:00,2024-04-15 00:00:00,NaN,arrendamientos,5254.384,335.892,36,Otras_Políticas,Formato de autorización de consulta parcialmen...,Incumple\n P & P,2 - Medio,No,No,Palmira,3,1,NaN,Análisis de carácter,Políticas,NaN,2024,NaN
3624,Nelly Munoz Vivas,ROO,Robinson Ocoro Centeno,Preferenci,2024-04-15 00:00:00,2024-04-15 00:00:00,2024-05-17 10:09:54.230000,arrendamientos,5254.384,335.892,36,Controles_Administrativos_de_Agencia,Deficiencias en el análisis de comité de crédito,Incumple\n P & P,2 - Medio,No,No,Palmira,3,1,NaN,Análisis de carácter,Control,NaN,2024,NaN


--> With the above, we can see that each observation is a different audit analysis or **audit finding**, even for the same client/account.

In [603]:
df_base.Cliente.sample(10)

309       Carlos Alberto Cardona Ramirez
1514               Farida  Franco Valero
2152    Yecid Javier Gallardo Villalobos
408        Yesica Milena Londono Vallejo
385            Edilson  Cardona Betancur
3753        Carolina  Santacruz Sandoval
3213       Leydi Johanna Vargas Gonzalez
3623       Monica Lizeth Valencia Cuaran
432               Yorleny  Gomez Escobar
1376           Teodolinda  Araque Araque
Name: Cliente, dtype: object

In [604]:
# df_base[df_base["Cliente"].str.contains("Simon Jose", na=False)]

--> After some explorations, we will assume that the client column doesnt have typos or gramatical errors, that affects in someway the data



**CONCLUSIONS** 


Taking in consideration the possible uses or goals for this project, we will take this considerations:
1. **This column isnt useful for predictions/inference of an anomaly detection or clasification model, but we will keep it only for reference and EDA**.
2. **The clients-credit in the dataset should be unique, so we will eliminate the duplicated observations for the same Client-credit combination**. We take in consideration the possibility of a same client to have more than one credit in different moments.

In [605]:
df_base = df_base.drop_duplicates(subset=["Cliente","Fecha desem", "Monto", "Cuota", "Plazo"]) #To have only one observation per client-credit

# rename Cliente column to avoid capital letters
df_base.rename(columns={"Cliente":"cliente"}, inplace=True)

In [606]:
df_base.shape

(3461, 26)

## Analista & Nombre analista

In [607]:
num_dif = df_base.Analista.nunique()
print(f'Number of different analyst: {num_dif}')

temp = df_base.Analista.value_counts()
temp = temp[temp > 1]
print(f'Number of analyst with more than one record: {len(temp)}')
print(f'\nAnalysts with more than one record:')
temp

Number of different analyst: 399
Number of analyst with more than one record: 382

Analysts with more than one record:


GEE    44
IOC    43
FPN    34
IMJ    31
CNZ    29
       ..
MTM     2
KGW     2
FTM     2
LVL     2
CRA     2
Name: Analista, Length: 382, dtype: int64

In [608]:
df_base.Analista.sample(5)

2637    HBM
1661    ICZ
409     FUN
534     EHW
3702    LNK
Name: Analista, dtype: object

In [609]:
df_base.Analista.apply(lambda x: len(x)).value_counts()

3    3461
Name: Analista, dtype: int64

--> All the analyst codes have 3 letters.

In [610]:
num_dif = df_base["Nombre analista"].nunique()
print(f'Number of different analyst"s name: {num_dif}')

temp = df_base["Nombre analista"].value_counts()
temp = temp[temp > 1]
print(f'Number of analyst"s name with more than one record: {len(temp)}')
print(f"\nAnalysts with more than one record:")
temp

Number of different analyst"s name: 399
Number of analyst"s name with more than one record: 379

Analysts with more than one record:


Aleatorio                         94
Gilberto Duque Duque              44
Nicolas  Suaza Pulgarin           43
Lindy Yureidy Areiza Osorio       34
Esmeralda  Guayara Medina         31
                                  ..
Silvia Yeraldine Mejia Rumbo       2
Marcela Paola Solano Fernandez     2
Mayerly Ducuara Viuche             2
Ruben Alejandro Vivas Gaspar       2
Juan Camilo Segura Palma           2
Name: Nombre analista, Length: 379, dtype: int64

**CONCLUSIONS**

1. **Both columns isnt valuable for our analysis nor modelling phase, but we will keep the analyst code for reference. For that reason, we will eliminate the analyst name.**
2. **We will rename the column Analista**.

In [611]:
df_base = df_base.drop(columns=["Nombre analista"])

#rename analista column to avoid capital letters
df_base = df_base.rename(columns={"Analista": "analista"})

## Tipo crédito

In [612]:
num_dif = df_base["Tipo crédito"].nunique()
print(f'Number of different credit types: {num_dif}')
print(f'Number of nan values: {df_base["Tipo crédito"].isna().sum()}')

temp = df_base["Tipo crédito"].value_counts()
print(f"\nCredit types with more than one record:")
temp

Number of different credit types: 5
Number of nan values: 202

Credit types with more than one record:


Nuevo           1385
Renovacion       816
Preferenci       469
Preferencial     321
Renovación       268
Name: Tipo crédito, dtype: int64

-- > There are a typo with the world "Preferencial" and "Renovacion". 

**CONCLUSIONS**

1. **We will rename the "Preferenci" values as "Preferencial".**
2. **We will rename the "Renovación" values as "Renovacion"**
3. **We will rename the nan values with "No_especificado"**.
4. **We will rename the column name to "tipo_credito"**
5. We consider this column an important one for further analysis

In [613]:
df_base = df_base.rename(columns={"Tipo crédito": "tipo_credito"})

In [614]:
df_base["tipo_credito"] = df_base["tipo_credito"].fillna("No_especificado")
df_base["tipo_credito"] = np.where(df_base["tipo_credito"] == "Renovación", "Renovacion", df_base["tipo_credito"])
df_base["tipo_credito"] = np.where(df_base["tipo_credito"] == "Preferenci", "Preferencial", df_base["tipo_credito"])
df_base["tipo_credito"].value_counts()

Nuevo              1385
Renovacion         1084
Preferencial        790
No_especificado     202
Name: tipo_credito, dtype: int64

## Fechas: de desembolso, visita analista y visita auditor

In [615]:
df_base[["Fecha desem","Visita_analista_crédito","Visita_auditor"]].dtypes

Fecha desem                object
Visita_analista_crédito    object
Visita_auditor             object
dtype: object

In [616]:
df_base[["Fecha desem","Visita_analista_crédito","Visita_auditor"]].sample(10, random_state=1)

,Fecha desem,Visita_analista_crédito,Visita_auditor
2431,2023-01-06 00:00:00,2023-01-02 00:00:00,2023-02-16 00:00:00
3342,2024-01-31 00:00:00,2024-01-30 00:00:00,2024-03-08 00:00:00
2107,2022-05-16 00:00:00,N/D,2023-08-16 00:00:00
1608,2022-06-13 00:00:00,2022-06-13 00:00:00,2023-01-13 00:00:00
3488,2024-03-27 00:00:00,NaN,2024-05-07 16:30:00
3905,2024-07-10 00:00:00,2024-07-05 00:00:00,2024-08-16 11:37:49.210000
968,31/03/2022,31/03/2022,00:00:00
2308,2023-07-22 00:00:00,2023-07-18 00:00:00,2023-07-28 08:50:00
1881,2023-03-30 00:00:00,2023-03-29 00:00:00,2023-05-09 00:00:00
3364,2024-02-29 00:00:00,2024-02-22 00:00:00,2024-03-13 00:00:00


--> Above we can see at least 6 different formats that these columns have. This is an unwanted problem.

In [617]:
df_base[["Fecha desem","Visita_analista_crédito","Visita_auditor"]].isna().sum()

Fecha desem                  3
Visita_analista_crédito    753
Visita_auditor             111
dtype: int64

In [618]:
df_base[(df_base["Visita_auditor"]=="00:00:00") | (df_base["Visita_auditor"]==datetime.time(0, 0))].shape

(126, 25)

--> We can note that there are some observations with dates like "00:00:00"

**CONCLUSIONS**  
  
For further analysys or feature engineering, we can keep the columns "Fecha desem" and "Visita_analista_credito".

1. **We will rename the columns to avoid capital letters and blank spaces.**
2. **We will standarize the columns to a date format, avoiding errors and missing values**.
3. **We will drop the observations with dates like "00:00:00".**
4. **We will eliminate the auditor visit column**, because in an hipotetic modelling phase, we cant know the auditor visit in advance.

In [619]:
# rename columns
df_base = df_base.rename(columns={"Visita_analista_crédito": "visita_analista_credito","Fecha desem": "fecha_desembolso"})

# drop observations with dates similar to 00:00:00
df_base = df_base[(df_base["Visita_auditor"]!="00:00:00") | (df_base["Visita_auditor"]!=datetime.time(0, 0))]

# Define a function to handle different dates formats
def convert_to_datetime(value):
    if pd.isna(value) or value in ["N/D", "00:00:00", "NAN"]:  # Handle missing or invalid values
        return pd.NaT
    try:
        return pd.to_datetime(value, dayfirst=True, errors='coerce')  # Convert to datetime and handle errors
    except:
        return pd.NaT  # Return NaT if it fails

# Apply function to both columns
df_base["fecha_desembolso"] = df_base["fecha_desembolso"].apply(convert_to_datetime)
df_base["visita_analista_credito"] = df_base["visita_analista_credito"].apply(convert_to_datetime)

# Drop column
df_base = df_base.drop(columns=["Visita_auditor"])

In [620]:
df_base[["fecha_desembolso","visita_analista_credito"]].isna().sum()

fecha_desembolso             4
visita_analista_credito    901
dtype: int64

In [621]:
df_base[["fecha_desembolso","visita_analista_credito"]].dtypes

fecha_desembolso           datetime64[ns]
visita_analista_credito    datetime64[ns]
dtype: object

In [622]:
df_base[["fecha_desembolso","visita_analista_credito"]].sample(10, random_state=1)

,fecha_desembolso,visita_analista_credito
2431,2023-01-06,2023-01-02
3342,2024-01-31,2024-01-30
2107,2022-05-16,NaT
1608,2022-06-13,2022-06-13
3488,2024-03-27,NaT
3905,2024-07-10,2024-07-05
968,2022-03-31,2022-03-31
2308,2023-07-22,2023-07-18
1881,2023-03-30,2023-03-29
3364,2024-02-29,2024-02-22


## Actividad

In [623]:
num_dif = df_base["Actividad"].nunique()
print(f'Number of different activities values: {num_dif}')
print(f'Number of nan values: {df_base["Actividad"].isna().sum()}')

temp = df_base["Actividad"].value_counts()
print(f"\nActivities with more than one record:")
temp[0:15]

Number of different activities values: 2364
Number of nan values: 52

Activities with more than one record:


venta de mercancia    57
tienda                53
arriendos             48
venta de comidas      29
maestro de obra       24
venta de ropa         21
Venta De Mercancia    20
arrendamientos        18
confeccion            17
construccion          14
venta de leche        14
venta de comida       14
confecciones          14
Tienda                13
venta de bebidas      13
Name: Actividad, dtype: int64

In [624]:
temp.sample(10, random_state=10)

venta de fritos                             6
peluquer?a marisol                          1
cl?nica de ropa gladys                      1
arriendos viviendas                         1
venta de tenis                              1
distribuidora mariajo embutidos             1
venta de calzado deportivo y ropa           1
mini market carzay                          1
venta mercancias y revistas por catalogo    1
cultio de fresa                             1
Name: Actividad, dtype: int64

--> We can see a lot of different activities, but also we can see that there are some values that are referring to the same, for example "Tienda" and "tienda".

--> Several activities have typos and errors. This could difficult the analysis or next phases.

**CONCLUSIONS**  

The activity variable could be very important for any EDA or modelling, but it has a lot of values, errors and should be very difficult to fix.  
Despite that, we will try to fix the Activity column in order to gather some information in next steps.  

1. **We will rename the variable column.**
2. **We will run some fixes and considerations to the Activity column**.
3. **and we will create a new Activity_v2 column to not drop the original one.**

In [625]:
# Function to clean Activity values
def clean_occupation(value):
    if pd.isna(value):  # Handle missing values
        return "no_especificado"
    value = value.strip()  # Remove spaces at the beginning and end
    value = unidecode(value)  # Remove accents
    value = value.lower()  # Convert to lowercase
    value = value.replace(" ", "_")  # Replace spaces with underscores
    return value

# Apply function to the column
df_base["Actividad"] = df_base["Actividad"].apply(clean_occupation)

# rename column
df_base = df_base.rename(columns={"Actividad": "actividad"})

## Monto, cuota & plazo

In [626]:
df_base[["Monto","Cuota","Plazo"]].dtypes

Monto    float64
Cuota     object
Plazo      int64
dtype: object

In [627]:
df_base[["Monto","Cuota","Plazo"]].isna().sum()

Monto    0
Cuota    0
Plazo    0
dtype: int64

In [628]:
df_base[(df_base["Cuota"]==0) | (df_base["Plazo"]==0) | (df_base["Monto"]==0)].shape

(12, 24)

--> We note that the "Cuota" variable is an object, also there isnt nan values.  

--> We also note that there are some observations with zero

In [629]:
df_base[["Monto","Cuota","Plazo"]].iloc[0:5]

,Monto,Cuota,Plazo
0,15069.268,792.642,36
1,402.000,209.228,626
2,1433.360,178.915,15
3,19213.296,991.582,42
4,1055.378,205.648,6


--> We note that the values of Monto & Cuota are divided by 1000

**CONCLUSIONS**

1. We will modify the column names to avoid capital letters.
2. **We will cast the Cuota column to float.** To do this, it is necessary to "clean" this column before cast it.
3. **We will multiply the Monto & Cuota by 1000.**
4. **We will drop the observations with a zero value in any of the three columns.**

In [630]:
# rename columns
df_base = df_base.rename(columns={"Monto": "monto", "Cuota": "cuota", "Plazo": "plazo"})

# eliminate characters that are not numbers in Cuota
df_base["cuota"] = df_base["cuota"].astype(str).str.replace(r'[^0-9.]', '', regex=True)

# turn empty strings into 0
df_base["cuota"] = np.where(df_base["cuota"] == "", "0", df_base["cuota"])

# convert to float
df_base["cuota"] = pd.to_numeric(df_base["cuota"], errors='coerce')

# multiply by 1000 to convert to thousands
df_base["cuota"] = round(df_base["cuota"] * 1000)
df_base["monto"] = round(df_base["monto"] * 1000)

# drop the observations with a zero value in any of the three columns.
df_base = df_base[(df_base["cuota"] != 0) & (df_base["plazo"] != 0) & (df_base["monto"] != 0)]

In [631]:
print(df_base[["monto","cuota","plazo"]].iloc[0:2])

df_base[["monto","cuota","plazo"]].dtypes


        monto     cuota  plazo
0  15069268.0  792642.0     36
1    402000.0  209228.0    626


monto    float64
cuota    float64
plazo      int64
dtype: object

## Categoria, hallazgo, tipo hallazgo, Riesgo

In [632]:
df_base[["Categoria","Hallazgo","Tipo hallazgo", "Riesgo"]].dtypes

Categoria        object
Hallazgo         object
Tipo hallazgo    object
Riesgo           object
dtype: object

In [633]:
df_base[["Categoria","Hallazgo","Tipo hallazgo", "Riesgo"]].isna().sum()

Categoria        0
Hallazgo         2
Tipo hallazgo    1
Riesgo           4
dtype: int64

In [634]:
df_base[["Categoria","Hallazgo","Tipo hallazgo", "Riesgo"]].sample(5,random_state=2)

,Categoria,Hallazgo,Tipo hallazgo,Riesgo
2054,Sin_Hallazgo,Sin hallazgo,Sin Hallazgo,4 - Otro
2551,Otras_Políticas,No cumple con los requisitos propios del crédito,Incumple\n P & P,3 - Bajo
1518,REPUTACIÓN,Referencias no contactadas por el analista,Fallas en la Metodología,2 - Medio
931,ESTABILIDAD,Negocio en sociedad,Incumple P & P,2 - Medio
529,Sin_Hallazgo,Sin hallazgo,Sin Hallazgo,4 - Otro


--> We can see that each column have different strings formats, for example, capital letters, blank spaces, patterns like "\n", etc.

Now, we will review each column further

### Categoria

In [635]:
num_dif = df_base["Categoria"].nunique()
print(f'Number of different Category values: {num_dif}')
print(f'Number of nan values: {df_base["Categoria"].isna().sum()}')

temp = df_base["Categoria"].value_counts()
print(f"\nCategory top:")
temp[0:]

Number of different Category values: 30
Number of nan values: 0

Category top:


Sin_Hallazgo                            1022
Otras_Políticas                          525
OTRAS_POLITICAS                          362
Limitación_en_el_Alcance                 214
LIMITACIÓN_EN_EL_ALCANCE                 196
ESTABILIDAD                              153
Políticas_Críticas                       121
CALIDAD_DE_DATOS                         109
Otros                                    109
Existencia_y_Continuidad_del_Negocio      90
FRAUDE_Y_PRÁCTICAS_INDEBIDAS              87
Estabilidad                               72
Reputación                                69
Calidad_de_Datos                          57
FLUJO_DE_CAJA                             48
POLITICAS_CRITICAS                        38
INCUMPLIMIENTO_AL_SAC                     36
Flujo_de_Caja                             30
EXISTENCIA_Y_CONTINUIDAD_DEL_NEGOCIO      24
Controles_Gestión_Comercial               23
REPUTACIÓN                                15
Habilidad_Empresarial                     13
Incumplimi

--> We find some values that are referred to the same but are wroten different.

### Hallazgo

In [636]:
num_dif = df_base["Hallazgo"].nunique()
print(f'Number of different Findings values: {num_dif}')
print(f'Number of nan values: {df_base["Hallazgo"].isna().sum()}')

temp = df_base["Hallazgo"].value_counts()
print(f"\nFindings top:")
temp[0:]

Number of different Findings values: 89
Number of nan values: 2

Findings top:


Sin hallazgo                                                                                                                  1022
Formato de vinculación con campos en blanco y/o errados                                                                        413
Formato de autorización de consulta parcialmente diligenciado                                                                  276
No se logró evidenciar la existencia del cliente y/o negocio                                                                   102
Diferencias en el arraigo del cliente                                                                                          100
Se evidencio existencia del cliente pero no se pudo confirmar el negocio                                                       100
Se evidenció existencia del cliente pero no se pudo confirmar el negocio                                                        85
Negocio en sociedad                                                                

--> This column is a description of the findings, is like a comments column. So, it is not necessary to modify a lot this column. Even, we could ignore this column.

### Tipo hallazgo

In [637]:
num_dif = df_base["Tipo hallazgo"].nunique()
print(f'Number of different finding types values: {num_dif}')
print(f'Number of nan values: {df_base["Tipo hallazgo"].isna().sum()}')

temp = df_base["Tipo hallazgo"].value_counts()
print(f"\nfinding types top:")
temp[0:]

Number of different finding types values: 10
Number of nan values: 1

finding types top:


Sin Hallazgo                1018
Incumple\n P & P             632
Fallas en la Metodología     570
Incumple P & P               470
Limitación                   410
Acto Irregular               222
Otros                        110
Incumple P&P                   9
Sin_Hallazgo                   3
Sin hallazgo                   2
Name: Tipo hallazgo, dtype: int64

--> "Limitacion" is referred that the audit cant be completed due to several reasons. This observations doesnt add value to the next steps.

### Riesgo

In [638]:
num_dif = df_base["Riesgo"].nunique()
print(f'Number of different Risk values: {num_dif}')
print(f'Number of nan values: {df_base["Riesgo"].isna().sum()}')

temp = df_base["Riesgo"].value_counts()
print(f"\nRisk top:")
temp[0:]

Number of different Risk values: 5
Number of nan values: 4

Risk top:


4 - Otro     1363
2 - Medio    1130
3 - Bajo      443
1 - Alto      348
No aplica     159
Name: Riesgo, dtype: int64

--> The "Riesgo" column is the cleanest one. It doesnt need cleaning steps.

Now lets check the "No aplica" level

In [639]:
df_base[df_base["Riesgo"]=="No aplica"]["Categoria"].value_counts()

LIMITACIÓN_EN_EL_ALCANCE    95
Sin_Hallazgo                64
Name: Categoria, dtype: int64

In [640]:
df_base[df_base["Riesgo"]=="No aplica"]["Hallazgo"].value_counts()

Sin hallazgo                                                                               64
No se logro evidenciar la existencia del cliente y/o negocio                               43
Se evidencio existencia del cliente pero no se pudo confirmar el negocio                   38
Se evidencio existencia del negocio pero no se pudo confirmar la información financiera    13
Name: Hallazgo, dtype: int64

In [641]:
df_base[df_base["Categoria"]=="Sin_Hallazgo"]["Riesgo"].value_counts()

4 - Otro     954
No aplica     64
Name: Riesgo, dtype: int64

--> The "No aplica" level have values without findings or with limitations to complete the audit.

----

### Conclusion

To clean this columns:

1. We will drop the few NA values for each column
2. **We will avoid capital letters and drop some patterns.**
3. **We will rename some values of 'Categoria' & 'Tipo hallazgo' columns in order to unify levels.**
4. We will rename the columns to avoid capital letters.
5. **We will drop the observations with "Tipo hallazgo" equal to "Limitacion"**.
6. Due to the previous step (#5), we will rename the level "No aplica" of the "Riesgo" column to "4 - otro"

In [642]:
# Drop nan values of the 4 columns
df_base = df_base.dropna(subset=["Categoria","Hallazgo","Tipo hallazgo", "Riesgo"])

# Function to clean the column values
def clean_string_column(value):
    if pd.isna(value):  # Handle missing values
        return "no_especificado"
    value = value.strip()  # Remove spaces at the beginning and end
    value = unidecode(value)  # Remove accents
    value = value.lower()  # Convert to lowercase
    return value

# Apply function to the 3 columns
df_base["Categoria"] = df_base["Categoria"].apply(clean_string_column)
df_base["Hallazgo"] = df_base["Hallazgo"].apply(clean_string_column)
df_base["Tipo hallazgo"] = df_base["Tipo hallazgo"].apply(clean_string_column)

# Combine some finding types values due to typos (incumple p&p)
df_base["Tipo hallazgo"] = df_base["Tipo hallazgo"].apply(lambda x: "incumple_p&p" if "incumple" in x else x)

# after review, there isnt more typos in "Categoria" column

# rename columns
df_base = df_base.rename(
    columns={
        "Tipo hallazgo": "tipo_hallazgo",
        "Riesgo": "riesgo",
        "Categoria": "categoria",
        "Hallazgo": "hallazgo",
    }
)

# drop observations with limitations (audit incomplete)
df_base=df_base[df_base["tipo_hallazgo"]!="limitacion"]

# Rename values "No aplica" of column Riesgo
df_base["riesgo"] = df_base["riesgo"].str.replace("No aplica","4 - Otro")

In [643]:
df_base[["categoria","hallazgo","tipo_hallazgo", "riesgo"]].sample(5,random_state=2)

,categoria,hallazgo,tipo_hallazgo,riesgo
504,sin_hallazgo,sin hallazgo,sin hallazgo,4 - Otro
2649,otras_politicas,formato de autorizacion de consulta parcialmen...,incumple_p&p,2 - Medio
695,calidad_de_datos,direccion del negocio no corresponde con la ve...,fallas en la metodologia,3 - Bajo
2673,otras_politicas,formato de vinculacion con campos en blanco y/...,incumple_p&p,2 - Medio
839,estabilidad,diferencias en el arraigo del cliente,fallas en la metodologia,3 - Bajo


In [644]:
df_base[["categoria","hallazgo","tipo_hallazgo", "riesgo"]].isna().sum()

categoria        0
hallazgo         0
tipo_hallazgo    0
riesgo           0
dtype: int64

In [645]:
df_base.tipo_hallazgo.value_counts()

incumple_p&p                1111
sin hallazgo                1020
fallas en la metodologia     570
acto irregular               222
otros                        110
Name: tipo_hallazgo, dtype: int64

In [646]:
df_base.riesgo.value_counts()

2 - Medio    1130
4 - Otro     1112
3 - Bajo      443
1 - Alto      348
Name: riesgo, dtype: int64

In [647]:
df_base.shape

(3033, 24)

## Calificacion cartera y Relaciones laborales

In [648]:
df_base["Calificación cartera"].value_counts()  

No    2803
Si     227
si       3
Name: Calificación cartera, dtype: int64

In [649]:
df_base["Relaciones Laborales"].value_counts()  

No    2309
Si     327
Name: Relaciones Laborales, dtype: int64

In [650]:
df_base[["Calificación cartera","Relaciones Laborales"]].isna().sum()  

Calificación cartera      0
Relaciones Laborales    397
dtype: int64

--> This columns have few typos and Nan values

**CONCLUSION**

1. Rename columns to avoid ccapital letters
2. We will change NA values to "no_especificado"
3. fix typos in Calificación cartera

In [651]:
# rename columns
df_base.rename(columns={"Calificación cartera": "calificacion_cartera", "Relaciones Laborales": "relaciones_laborales"}, inplace=True)

# NA values to "no_especificado"
df_base["calificacion_cartera"] = df_base["calificacion_cartera"].fillna("no_especificado")
df_base["relaciones_laborales"] = df_base["relaciones_laborales"].fillna("no_especificado")

# fix typos
df_base["calificacion_cartera"] = df_base["calificacion_cartera"].apply(lambda x: "Si" if "si" in x else x)


## Oficina

In [652]:
num_dif = df_base["Oficina "].nunique()
print(f'Number of different "Oficina" values: {num_dif}')
print(f'Number of nan values: {df_base["Oficina "].isna().sum()}')

temp = df_base["Oficina "].value_counts()
print(f"\n'Oficina' top:")
temp

Number of different "Oficina" values: 63
Number of nan values: 0

'Oficina' top:


Dosquebradas              177
Ibague                    130
Fonsecca                  125
Tulua                      91
Chaparral                  79
Santa Rosa                 77
Facatativa                 70
Laureano Gómez             60
Independencia              57
Magangue                   56
Pasto                      55
Tulua Calle 25             54
Sincelejo                  53
Engativa                   52
Poblado                    51
Maicao                     49
Palmira                    49
Sur                        49
Pereira Lagos              48
Pasto Las Lunas            48
El Espinal                 47
Cartagena del Chaira       47
Santa Marta                46
San Juan del CesAR         45
Armenia Sur                45
La Plata                   45
Jamundí                    45
Santander de Quilichao     45
Soledad                    45
La Unión                   44
Corozal                    44
Calima                     44
Ipiales                    43
Riohacha  

--> This column seems without problems nor typos.

**CONCLUSION**

1. Change column name.
2. avoid capital letters and blank spaces.

In [653]:
# Rename column Oficina
df_base = df_base.rename(columns={"Oficina ": "oficina"})

# avoid capital letters and blank spaces
df_base["oficina"] = df_base["oficina"].apply(lambda x: x.strip().lower())

## Zona, regional

In [654]:
num_dif = df_base["zona "].nunique()
print(f'Number of different "zona" values: {num_dif}')
print(f'Number of nan values: {df_base["zona "].isna().sum()}')

temp = df_base["zona "].value_counts()
print(f"\n'zona' top:")
temp

Number of different "zona" values: 17
Number of nan values: 0

'zona' top:


12    390
11    340
13    304
3     252
7     220
4     216
1     201
15    197
8     164
6     137
5     131
14    121
2      94
10     92
16     81
17     56
9      37
Name: zona , dtype: int64

In [655]:
num_dif = df_base["Regional"].nunique()
print(f'Number of different "zona" values: {num_dif}')
print(f'Number of nan values: {df_base["Regional"].isna().sum()}')

temp = df_base["Regional"].value_counts()
print(f"\n'zona' top:")
temp

Number of different "zona" values: 5
Number of nan values: 0

'zona' top:


3    824
4    757
1    653
5    502
2    297
Name: Regional, dtype: int64

--> Both columns also seems without problems

**CONCLUSION**

1. We will only change the column names.

In [656]:
# change column names
df_base = df_base.rename(columns={"zona ": "zona", "Regional": "regional"})

## Validacion unico

In [657]:
num_dif = df_base["Validación unico"].nunique()
print(f'Number of different "validacion unico" values: {num_dif}')
print(f'Number of nan values: {df_base["Validación unico"].isna().sum()}')

temp = df_base["Validación unico"].value_counts()
print(f"\n'validacion unico' top:")
temp

Number of different "validacion unico" values: 2
Number of nan values: 2821

'validacion unico' top:


ok          184
repetido     28
Name: Validación unico, dtype: int64

--> This column seems like a validation column to check some "repeated" observations based on something that we currently dont know.

**CONCLUSION** 
1. For now, we will drop the observations with "repetido" and also drop the column.

In [658]:
# drop observations with "repetido" value and drop column
df_base = df_base[df_base["Validación unico"] != "repetido"]
df_base = df_base.drop(columns=["Validación unico"])

## Tipo analisis

In [659]:
num_dif = df_base["Tipo analisis "].nunique()
print(f'Number of different "Tipo analisis" values: {num_dif}')
print(f'Number of nan values: {df_base["Tipo analisis "].isna().sum()}')

temp = df_base["Tipo analisis "].value_counts()
print(f"\n'Tipo analisis' top:")
temp

Number of different "Tipo analisis" values: 5
Number of nan values: 1353

'Tipo analisis' top:


Análisis de carácter     978
Sin hallazgo             566
Otros                     56
Analisis Cuantitativo     48
Analisis de carácter       4
Name: Tipo analisis , dtype: int64

--> This column have a lot of NA values and is no clear how this information could add value to our next steps.

**CONCLUSIONS**
1. We will drop this column due to the lack of valuable information

In [660]:
#drop column Tipo analisis
df_base = df_base.drop(columns=["Tipo analisis "])

## Clasificacion analisis

In [661]:
num_dif = df_base["Clasificac analisis"].nunique()
print(f'Number of different "Clasificacion analisis" values: {num_dif}')
print(f'Number of nan values: {df_base["Clasificac analisis"].isna().sum()}')

temp = df_base["Clasificac analisis"].value_counts()
print(f"\n'Clasificacion analisis' top:")
temp

Number of different "Clasificacion analisis" values: 12
Number of nan values: 1353

'Clasificacion analisis' top:


Políticas                      617
Sin hallazgo                   566
Estabilidad                    103
Alerta de fraude               101
Reputación                      76
Calidad de datos                65
Flujo de caja                   48
Otros                           42
Quejas                          13
Habilidad empresarial           12
Control                          8
Seguridad de la información      1
Name: Clasificac analisis, dtype: int64

--> This column have a lot of NA values and we think that this column data is very similar to the column "Categoria".

**CONCLUSIONS**
1. We will drop this column due that its data is similar to other column that have more information

In [662]:
#drop column Tipo analisis
df_base = df_base.drop(columns=["Clasificac analisis"])

## Estado

In [663]:
num_dif = df_base["Estado"].nunique()
print(f'Number of different status values: {num_dif}')
print(f'Number of nan values: {df_base["Estado"].isna().sum()}')

temp = df_base["Estado"].value_counts()
print(f"\nStatus top:")
temp

Number of different status values: 6
Number of nan values: 626

Status top:


Normal                            1078
Cancelada                          983
Castigo                            207
Modificado                          92
Cobro Judicial                      13
Reestructurado                       6
Name: Estado, dtype: int64

--> This columns seems to have valuable information about the credit status. Doesnt have typos but several NA values.

**CONCLUSIONS**
1. We will rename the column to avoid capital letters.
2. We will fill NA values with "no_especificado".
3. We will clean a little more the column to avoid capital letters nor blank spaces at the beginning or the last of the string.

In [664]:
#rename column Estado
df_base = df_base.rename(columns={"Estado": "estado"})

# fill NA with no_especificado
df_base["estado"] = df_base["estado"].fillna("no_especificado")

# eliminate capital letters and blank spaces
df_base["estado"] = df_base["estado"].apply(lambda x: x.strip().lower())

## Año

In [665]:
num_dif = df_base["Año"].nunique()
print(f'Number of different year values: {num_dif}')
print(f'Number of nan values: {df_base["Año"].isna().sum()}')

temp = df_base["Año"].value_counts()
print(f"\n Year top:")
temp

Number of different year values: 3
Number of nan values: 0

 Year top:


2022    1353
2023    1026
2024     626
Name: Año, dtype: int64

--> Seems without problems

**CONCLUSIONS**
1. We will only rename the column


In [666]:
# rename the column year
df_base = df_base.rename(columns={"Año": "year"})

## Tipo

In [667]:
num_dif = df_base["Tipo"].nunique()
print(f'Number of different type values: {num_dif}')
print(f'Number of nan values: {df_base["Tipo"].isna().sum()}')

temp = df_base["Tipo"].value_counts()
print(f"\n type top:")
temp

Number of different type values: 3
Number of nan values: 626

 type top:


Otros Hallazgos       1483
Sin Hallazgo           748
Afecta Estabilidad     148
Name: Tipo, dtype: int64

--> This column seems without typos but have several NA values. Right now, is not clear what is the information that this column have but we will keep it by now.

**CONCLUSIONS**
1. We will rename the column.
2. We will fill the NA values with "no_especificado"
3. We will avoid capital letters and blank spaces

In [668]:
#rename the column
df_base = df_base.rename(columns={"Tipo": "tipo"})

#fill NA with no_especificado
df_base["tipo"] = df_base["tipo"].fillna("no_especificado")

#avoid capital letters and blank spaces
df_base["tipo"] = df_base["tipo"].apply(lambda x: x.strip().lower())

# Temporal FE (new variables)

In [669]:
df_base['riesgo'].unique()

array(['1 - Alto', '4 - Otro', '3 - Bajo', '2 - Medio'], dtype=object)

In [670]:
# Create new column riesgo, labeling the values to numbers (integers)

df_base["riesgo_int"] = df_base["riesgo"].replace({
    "1 - Alto": 1,
    "2 - Medio": 2,
    "3 - Bajo": 3,
    "4 - Otro": 4
})

In [671]:
df_base["riesgo_int"].value_counts()

2    1117
4    1112
3     430
1     346
Name: riesgo_int, dtype: int64

In [672]:
# Function to combine some Activity values to create macro-activities
def combine_occupation(value):
    if any(char in value for char in ["no_especificado"]):
        return "no_especificado"
    if any(char in value for char in ["ambulant"]):
        return "comercio_ambulante"
    elif any(
        char in value
        for char in [
            "agrico",
            "agricultura",
            "agro",
            "agricultor",
            "culti",
            "ganad",
            "cerdo",
            "pollo",
            "vaca" "animal",
            "cr?",
            "cria",
            "platano",
            "banano",
        ]
    ):
        return "agricultura_ganaderia_y_afines"
    elif any(
        char in value
        for char in [
            "comida",
            "restau",
            "alimen",
            "condimento",
            "cafe",
            "caf?",
            "frit",
            "plato",
            "frut",
            "fruv",
            "pan",
            "helad",
            "arepa",
            "empana",
            "piz",
            "leche",
        ]
    ):
        return "sector_alimenticio"
    elif any(char in value for char in ["arriendo", "arren", "inmobi", "alquil"]):
        return "arriendos_alquiler_e_inmobiliarios"
    elif any(char in value for char in ["obra", "const", "interior"]):
        return "construccion_obras_y_afines"
    elif any(
        char in value for char in ["confecc", "modist", "sastr", "costur", "prenda"]
    ):
        return "confeccion_y_afines"
    elif any(
        char in value for char in ["venta", "vta", "comercio", "mercancia", "comerci"]
    ):
        return "comercio_y_ventas_general"
    elif any(char in value for char in ["tienda", "almac", "supermer", "abarro"]):
        return "tiendas_y_almacenes_varios"
    elif any(
        char in value
        for char in [
            "mecan",
            "autom",
            "metal",
            "tecnico",
            "t?cni",
            "electri",
            "soldad",
            "pint",
            "laton",
            "taller",
            "manteni",
            "reparaci",
            "arregl",
            "ebanist",
            "herr",
            "plome",
            "artesan",
        ]
    ):
        return "oficios_tecnicos_y_manuales"
    elif any(
        char in value for char in ["transp", "trasp", "taxi", "bus", "vehicu", "moto"]
    ):
        return "transporte_y_vehiculos"
    elif any(
        char in value
        for char in [
            "manicur",
            "peluq",
            "bell",
            "unas",
            "u?as",
            "spa",
            "pein",
            "estili",
            "maquill",
            "barber",
            "masaj",
            "depil",
            "esteti",
        ]
    ):
        return "belleza_y_estetica"
    elif any(
        char in value for char in ["enferm", "medic", "salud", "drog", "farm", "hospi"]
    ):
        return "salud_y_afines"
    elif any(
        char in value
        for char in ["educ", "escol", "refuer", "univ", "coleg", "clase", "docen"]
    ):
        return "servicios_educativos"
    elif any(char in value for char in ["aseo", "limpieza", "aseador"]):
        return "servicios_de_limpieza"
    # elif any(
    #     char in value
    #     for char in ["eria", "aria", "vario", "guada", "miscel", "publici"]
    # ):
    #     return "otros_comercios_servicios_oficios"
    else:
        return "otros"


df_base["actividad_v2"] = df_base["actividad"].apply(combine_occupation)

# To check all the activities that have "venta" or "vta" or "comercio" or "mercancia" or "comerci" in the name
# pd.Series([x for x in df_base["actividad"] if any(char in x for char in ["venta", "vta", "comercio","mercancia","comerci"])]).unique()

In [673]:
num_dif = df_base["actividad_v2"].nunique()
print(f'Number of different activities (Version2) values: {num_dif}')
print(f'Number of nan values: {df_base["actividad_v2"].isna().sum()}')

temp = df_base["actividad_v2"].value_counts()
print(f"\nActivities (Version2) with more than one record:")
temp

Number of different activities (Version2) values: 16
Number of nan values: 0

Activities (Version2) with more than one record:


otros                                 666
comercio_y_ventas_general             648
sector_alimenticio                    408
tiendas_y_almacenes_varios            199
agricultura_ganaderia_y_afines        194
oficios_tecnicos_y_manuales           169
belleza_y_estetica                    156
confeccion_y_afines                   142
arriendos_alquiler_e_inmobiliarios    127
construccion_obras_y_afines            97
transporte_y_vehiculos                 69
no_especificado                        51
salud_y_afines                         36
servicios_educativos                   17
comercio_ambulante                     15
servicios_de_limpieza                  11
Name: actividad_v2, dtype: int64

# Final result

In [590]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3005 entries, 0 to 3918
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   cliente                  3004 non-null   object        
 1   analista                 3005 non-null   object        
 2   tipo_credito             3005 non-null   object        
 3   fecha_desembolso         3002 non-null   datetime64[ns]
 4   visita_analista_credito  2194 non-null   datetime64[ns]
 5   actividad                3005 non-null   object        
 6   monto                    3005 non-null   float64       
 7   cuota                    3005 non-null   float64       
 8   plazo                    3005 non-null   int64         
 9   categoria                3005 non-null   object        
 10  hallazgo                 3005 non-null   object        
 11  tipo_hallazgo            3005 non-null   object        
 12  riesgo                   3005 non-

In [591]:
# move the actividad_v2 column next to actividad
df_base = df_base[["cliente", "analista", "tipo_credito", "actividad", "actividad_v2", "monto", "cuota", "plazo", "fecha_desembolso", "visita_analista_credito", "categoria", "hallazgo", "tipo_hallazgo", "calificacion_cartera", "relaciones_laborales", "oficina", "zona", "regional", "estado", "year", "tipo","riesgo","riesgo_int"]]

In [592]:
df_base.sample(5)

,cliente,analista,tipo_credito,actividad,actividad_v2,monto,cuota,plazo,fecha_desembolso,visita_analista_credito,categoria,hallazgo,tipo_hallazgo,calificacion_cartera,relaciones_laborales,oficina,zona,regional,estado,year,tipo,riesgo,riesgo_int
1225,Hernandez Agudelo Oscar ...,CXF,No_especificado,venta_de_frutas_y_verduras,sector_alimenticio,800000.0,168168.0,8,2022-05-28,2022-05-27,otras_politicas,formato de vinculacion con campos en blanco y/...,incumple_p&p,No,no_especificado,tulua calle 25,3,1,cancelada,2022,otros hallazgos,2 - Medio,2
3901,Carlos Mario Lopera Eusse,JPR,Preferencial,video_juegos,otros,2558540.0,277413.0,18,2024-07-06,2024-07-03,otras_politicas,formato de vinculacion con campos en blanco y/...,incumple_p&p,No,No,buenaventura,4,1,no_especificado,2024,no_especificado,2 - Medio,2
3490,Jose Napoleon Garcia Sotomayor,NBP,Renovacion,venta_de_mercanc?as_ropa,comercio_y_ventas_general,12157376.0,978704.0,24,2024-03-27,NaT,sin_hallazgo,sin hallazgo,sin hallazgo,No,No,maicao,15,4,no_especificado,2024,no_especificado,4 - Otro,4
2244,Luz Esneda Ortiz Gonzalez,KAU,Renovacion,servicios_domesticos,otros,3175976.0,252439.0,24,2023-04-25,2023-04-25,flujo_de_caja,diferencias en flujos de caja que afectan la l...,fallas en la metodologia,Si,No,rionegro,8,3,cancelada,2023,otros hallazgos,2 - Medio,2
2755,Gloriceth Tarras Urango,LGA,Nuevo,venta_de_cerdo_y_carima?olas,agricultura_ganaderia_y_afines,2178200.0,235048.0,18,2023-11-17,2023-11-17,otras_politicas,formato de vinculacion con campos en blanco y/...,incumple_p&p,No,No,cereté,15,5,normal,2023,otros hallazgos,2 - Medio,2


# Export clean data

In [595]:
# # Export df_base to output DIR --- AWARE-ONLY WHEN NEEDED
# OUTPUT_FILE = os.path.join(OUTPUT_DIR, "hallazgos_clean.xlsx")
# df_base.to_excel(OUTPUT_FILE)